In [1]:
# Importamos librerias necesarias
import numpy as np

import torch

from tqdm import tqdm
from classical_shadow import sample_measurements, NQS, batches, nll_batch, random_state
from tomography import pauli_basis, simulate_expectation, standard_tomography, fidelity

In [2]:
error = 0.02
n_exp = 500
psi_true = random_state()

train_data = sample_measurements(psi=psi_true, M=n_exp, error = error)

In [3]:
model = NQS()
opt = torch.optim.Adam(model.parameters(), lr=1e-2)
sch = torch.optim.lr_scheduler.StepLR(opt, step_size=50, gamma=0.5)

In [4]:
epochs = 500
batch = 64

In [5]:
# Step 4: training loop
for epoch in range(1, epochs + 1):
    model.train()  # set model to training mode
    
    for batch_data in batches(train_data, batch):
        # Compute negative log-likelihood loss for the current batch
        loss = nll_batch(model, batch_data)

        # Gradient step
        opt.zero_grad()
        loss.backward()
        opt.step()

    # Update learning rate schedule
    sch.step()

    # Show the loss
    if (epoch % 50 == 0) or (epoch == 1):
        print(f"Epoch {epoch} || Loss {loss.item()}")

Epoch 1 || Loss 2.7079617977142334
Epoch 50 || Loss 0.9937571287155151
Epoch 100 || Loss 0.926230788230896
Epoch 150 || Loss 0.8997865319252014
Epoch 200 || Loss 0.8844689726829529
Epoch 250 || Loss 0.8751189708709717
Epoch 300 || Loss 0.8694041967391968
Epoch 350 || Loss 0.8659293055534363
Epoch 400 || Loss 0.8638286590576172
Epoch 450 || Loss 0.8625679016113281
Epoch 500 || Loss 0.8618176579475403


In [6]:
rho_true = psi_true.detach().numpy()
rho_true = np.kron(rho_true, rho_true.conj().T).reshape(2, 2)

rho_pred = model.density_matrix().detach().numpy()
rho_pred = 1/2 * ( rho_pred + rho_pred.conj().T )
rho_pred = rho_pred / np.trace(rho_pred)

In [7]:
fidelity(rho = rho_true, sigma = rho_pred)

0.957472991351589